# 0L Contributors payments data 
## Generate payments data
### Pre-requisites
- access to well formed payments.json files
- a correctly configured config.py file
### TODO
- make search more efficient and intelligent

### Imports and loading variables

In [3]:
import requests
import json
import os

from config import Config

f = open(Config.CONTRIBUTORS_FILEPATH)
contributors = json.load(f)

# extract all addresses to be evaluated
address_list = [c for c in contributors.keys()]

print(address_list)

['5F8AC83A9B3BF2EFF20A6C16CD05C111', '5DB97DD068263449C8EB369DC2312AFA', 'E8DF1AFD258A10BD0392E1A8D274CC81']


### Get payments and generate the payment.json file
Here we will use an account list, generated from the contributors.json file, to iterate the accounts and fetch transactions which are relevant for us. On the fly, we are fetching on-chain transaction data that can be linked to real world data and generate a payments.json file which will have place holders for custom information linked to the payments.

In [2]:
# result list of all transactions, wrapped by mission data
result = []

# get all payments for address list between the given block height
for addr in address_list:
    
    end_not_reached = True
    start = 0
    limit = 1000
    while end_not_reached:
        url = f"https://0lexplorer.io/api/proxy/node/events?address={addr}&start={start}&limit={limit}"
        response = requests.get(url)
        response_json = response.json()
        if "result" in response_json:
            payload = response_json["result"]
            # check the payload for data of interest
            for tx in reversed(payload):
                if tx['transaction_version'] >= Config.START_BLOCK:
                    if tx['transaction_version'] <= Config.END_BLOCK and \
                       tx['data']['type'] in ['receivedpayment']:
                        # create payment entry
                        payment = {
                            "payment-round": f"{Config.PAYMENT_ROUND}",
                            "contributor": f"{addr}",
                            "workgroup_id": "MANUAL INPUT",     # analytics-wg, ... 
                            "assignment_ref": "MANUAL INPUT",   # URL to airtable of custom ID?
                            "payment_type": "MANUAL INPUT",     # work-related, bonus, rectification, ...
                            "description": "MANUAL INPUT",      # Short description for reporting
                            "contributor_role": "MANUAL INPUT", # broker, developer, analyst, ...
                            "reviewer": "MANUAL INPUT",         # broker role?
                            "tx": tx,
                            "artifacts":["MANUAL INPUT"],       # Links to work delivered if applicable
                            }
                        result.append(payment)
                else:
                    break

            if len(payload) < 1000:
                end_not_reached = False
            else:
                start += 1000
                limit += 1000
        else:
            end_not_reached = False
    
print(result)
        

[]


### Write payments data to json file 

In [3]:
fq_filename = f"{Config.PAYMENTS_DIR}/payments_{Config.PAYMENT_ROUND}.json"

# write result pretty-printed to a file
payment_file = open(fq_filename, "w")
payment_file.write(json.dumps(result, indent=4, sort_keys=True))
payment_file.close()

print(f"{fq_filename} created...")

/home/user/projects/ol-analytical-research/assets/generated/payments/payments_PR20221001_002.json created...
